In [46]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
from functions import genres_keywords,cast,director_name,collapse,stem
from model_preparation import word2vector, recommend

In [2]:
movies=pd.read_csv('Data/movies.csv')
credits=pd.read_csv('Data/credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
movies=movies.merge(credits,on='title')

In [6]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
movies.shape

(4809, 23)

In [8]:
df1=movies[['movie_id','genres','keywords','title','overview','cast','crew']]
df1.head(1)

,movie_id,genres,keywords,title,overview,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


#### Preprocessing

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   genres    4806 non-null   object
 2   keywords  4806 non-null   object
 3   title     4806 non-null   object
 4   overview  4806 non-null   object
 5   cast      4806 non-null   object
 6   crew      4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.4+ KB


In [13]:
df1.dropna(inplace=True)

C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\3614008390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(inplace=True)


In [14]:
df1.head(1)

,movie_id,genres,keywords,title,overview,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [26]:
df1['genres']=df1['genres'].apply(genres_keywords)
df1['keywords']=df1['keywords'].apply(genres_keywords)
df1['cast']=df1['cast'].apply(cast)
df1['Director']=df1['crew'].apply(director_name)

C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\3444084614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['genres']=df1['genres'].apply(genres_keywords)
C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\3444084614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['keywords']=df1['keywords'].apply(genres_keywords)
C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\3444084614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [27]:
df1.head(1)

,movie_id,genres,keywords,title,overview,cast,crew,Director
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[James Cameron]


In [33]:
df1.genres=df1.genres.apply(collapse)
df1.keywords=df1.keywords.apply(collapse)
df1.cast=df1.cast.apply(collapse)
df1.Director=df1.Director.apply(collapse)


C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\2098430038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.genres=df1.genres.apply(collapse)
C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\2098430038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.keywords=df1.keywords.apply(collapse)
C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\2098430038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [34]:
df1.head()

,movie_id,genres,keywords,title,overview,cast,crew,Director
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[JamesCameron]
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",[GoreVerbinski]
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,A cryptic message from Bond’s past sends him o...,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",[SamMendes]
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[ChristianBale, MichaelCaine, GaryOldman, Anne...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",[ChristopherNolan]
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"John Carter is a war-weary, former military ca...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",[AndrewStanton]


In [35]:
df1['overview']=df1.overview.apply(lambda x:x.split())

C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\271909524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['overview']=df1.overview.apply(lambda x:x.split())


In [36]:
df1['Tags']=df1['genres'] + df1['keywords'] + df1['overview'] + df1['cast'] + df1['Director']

C:\Users\Revan\AppData\Local\Temp\ipykernel_19408\1127814290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Tags']=df1['genres'] + df1['keywords'] + df1['overview'] + df1['cast'] + df1['Director']


In [39]:
new_df=df1.drop(['genres','keywords','overview','cast','crew','Director'],axis=1)
new_df.head(1)

,movie_id,title,Tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."


In [40]:
new_df['Tags'] = new_df['Tags'].apply(lambda x: " ".join(x))
new_df.head()

,movie_id,title,Tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...


In [41]:
new_df['Tags']=new_df['Tags'].apply(lambda x: x.lower())
new_df.head(2)

,movie_id,title,Tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...


In [44]:
new_df['Tags']=new_df['Tags'].apply(stem)

In [45]:
new_df.head(1)

,movie_id,title,Tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...


In [47]:
similarity=word2vector(new_df)
similarity

array([[1.        , 0.08108108, 0.08718573, ..., 0.04604093, 0.        ,
        0.        ],
       [0.08108108, 1.        , 0.05812382, ..., 0.02302046, 0.        ,
        0.02507061],
       [0.08718573, 0.05812382, 1.        , ..., 0.02475369, 0.        ,
        0.        ],
       ...,
       [0.04604093, 0.02302046, 0.02475369, ..., 1.        , 0.040008  ,
        0.04270814],
       [0.        , 0.        , 0.        , ..., 0.040008  , 1.        ,
        0.08714204],
       [0.        , 0.02507061, 0.        , ..., 0.04270814, 0.08714204,
        1.        ]])

In [48]:
recommended=recommend('Avatar',new_df,similarity)
recommended

['Aliens vs Predator: Requiem',
 'Independence Day',
 'Falcon Rising',
 'Battle: Los Angeles',
 'Titan A.E.']

In [58]:
movies_ds=pd.DataFrame(new_df[['movie_id','title']],columns=['movie_id','title'])
movies_ds.head(1)

,movie_id,title
0,19995,Avatar


In [52]:
import pickle

In [59]:
with open('API\movies_dataset.pickle','wb')as f:
    pickle.dump(movies_ds,f)

In [54]:
with open('API\similarity.pickle','wb')as f:
    pickle.dump(similarity,f)